In [1]:
import numpy as np
import pandas as pd
from ta.momentum import StochasticOscillator
from FinMind import strategies
from FinMind.data import DataLoader
from FinMind.strategies.base import Strategy
from FinMind.data import FinMindApi
import time
import sys,os
import requests

stock_id_ignored = ['0054', '0058', '0059', '0060', '00625K', '00643K', '00649', '00658L', '00659R', '00667', '00672L', '00677U', '00691R', '00698L', '00699R', '00704L', '00705R', '00716R', '00729R', '00732', '00742', '00743', '00766L', '00767', '00774B', '00774C', '00776', '0080', '0081', '008201', '00866', '00906', '00938', '00943', '00944', '00945B', '00946', '00947', '00949', '00951', '00952', '00953B', '00954', '00956', '00960', '00961', '00963', '00964', '00965']
stock_id_available = ['0050', '0051', '0052', '0053', '0055', '0056', '0057', '0061', '006203', '006204', '006205', '006206', '006207', '006208', '00631L', '00632R', '00633L', '00634R', '00635U', '00636', '00636K', '00637L', '00638R', '00639', '00640L', '00641R', '00642U', '00643', '00645', '00646', '00647L', '00648R', '00650L', '00651R', '00652', '00653L', '00654R', '00655L', '00656R', '00657', '00657K', '00660', '00661', '00662', '00663L', '00664R', '00665L', '00666R', '00668', '00668K', '00669R', '00670L', '00671R', '00673R', '00674R', '00675L', '00676R', '00678', '00680L', '00681R', '00682U', '00683L', '00684R', '00685L', '00686R', '00688L', '00689R', '00690', '00692', '00693U', '00700', '00701', '00702', '00703', '00706L', '00707R', '00708L', '00709', '00710B', '00711B', '00712', '00713', '00714', '00715L', '00717', '00728', '00730', '00731', '00733', '00735', '00736', '00737', '00738U', '00739', '00752', '00753L', '00757', '00762', '00763U', '00770', '00771', '00775B', '00783', '00830', '00850', '00851', '00852L', '00861', '00865B', '00875', '00876', '00878', '00881', '00882', '00885', '00891', '00892', '00893', '00894', '00895', '00896', '00897', '00898', '00899', '00900', '00901', '00902', '00903', '00904', '00905', '00907', '00908', '00909', '00910', '00911', '00912', '00913', '00915', '00916', '00917', '00918', '00919', '00920', '00921', '00922', '00923', '00924', '00925', '00926', '00927', '00929', '00930', '00932', '00934', '00935', '00936', '00939', '00940', '00941']

In [2]:
class Kd2(Strategy):
    """
    url: "https://www.mirrormedia.mg/story/20180719fin012/"
    summary:
        網路上常見的 kd 交易策略
        日KD 80 20
        日K線 <= 20 進場
        日K線 >= 80 出場
    """

    k_days = 9
    kd_upper = 80
    kd_lower = 30

    def create_trade_sign(self, stock_price: pd.DataFrame, additional_dataset_obj) -> pd.DataFrame:
        stock_price = stock_price.sort_values("date")
        kd = StochasticOscillator(
            high=stock_price["max"],
            low=stock_price["min"],
            close=stock_price["close"],
            n=self.k_days,
        )
        rsv_ = kd.stoch().fillna(50)
        _k = np.zeros(stock_price.shape[0])
        _d = np.zeros(stock_price.shape[0])
        for i, r in enumerate(rsv_):
            if i == 0:
                _k[i] = 50
                _d[i] = 50
            else:
                _k[i] = _k[i - 1] * 2 / 3 + r / 3
                _d[i] = _d[i - 1] * 2 / 3 + _k[i] / 3

        stock_price["K"] = _k
        stock_price["D"] = _d
        stock_price.index = range(len(stock_price))
        stock_price["signal"] = 0
        stock_price.loc[stock_price["K"] <= self.kd_lower, "signal"] = 1
        stock_price.loc[stock_price["K"] >= self.kd_upper, "signal"] = -1
        return stock_price


In [3]:
# update token from a file
token_path = "./token"
token = ""
with open(token_path, "r") as fp:
    token = fp.read()
    token = token.replace("\n", "")
if len(token) <= 100:
    print("ERROR: need a token from FinMind")
print(len(token))

# get account status
url = "https://api.web.finmindtrade.com/v2/user_info"
payload = {
    "token": token+"1",
}
resp = requests.get(url, params=payload)
print("使用次數:", resp.json()["user_count"])  # 使用次數
print("api 使用上限", resp.json()["api_request_limit"])  # api 使用上限
print(resp.status_code)
print(resp.json())


176
使用次數: 0
api 使用上限 600
200
{'msg': 'success', 'status': 200, 'email_verify': True, 'user_id': 'raulhsiao', 'email': 'raulhsiao@gmail.com', 'level': 1, 'end_date': '', 'user_count': 0, 'api_request_limit': 600, 'api_request_limit_hour': 600, 'api_request_limit_day': 600, 'level_title': 'Free', 'is_old_user': 0, 'BackerInfo': {'subscription_begin_date': '', 'subscription_expired_date': '', 'api_request_limit': 0, 'msg': '', 'status_code': 200}, 'SponsorInfo': {'subscription_begin_date': '', 'subscription_expired_date': '', 'api_request_limit': 0, 'msg': '', 'status_code': 200}, 'SponsorProInfo': {'subscription_begin_date': '', 'subscription_expired_date': '', 'api_request_limit': 0, 'msg': '', 'status_code': 200}}


In [4]:
api = DataLoader()
api.login_by_token(api_token=token)

fapi = FinMindApi()
fapi.login_by_token(token)
print(fapi.api_usage_limit)

stock_id_available = []

tw_stock_info_df = api.taiwan_stock_info()
stock_list_df = tw_stock_info_df.loc[tw_stock_info_df["industry_category"]=="ETF"]
for index,row in stock_list_df.iterrows():
    stock_id = row["stock_id"]
    stock_name = row["stock_name"]
    
    if stock_id in stock_id_ignored:
        continue
        
    df = api.taiwan_stock_daily(
        stock_id=row["stock_id"],
        start_date='2024-04-02',
        end_date='2024-04-30'
    )
    if df.size > 0:
        # print(row["stock_id"], row["stock_name"], row["date"], df.size)
        stock_id_available.append(row["stock_id"])
    else :
        print("BYPASS", )
        stock_id_ignored.append(row["stock_id"])
        
    # if index > 15: break

2025-01-21 11:55:41.359 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockInfo, data_id: 


600


2025-01-21 11:55:41.969 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 0050
2025-01-21 11:55:42.296 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 0051
2025-01-21 11:55:42.532 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 0052
2025-01-21 11:55:42.754 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 0053
2025-01-21 11:55:42.977 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 0055
2025-01-21 11:55:43.203 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 0056
2025-01-21 11:55:43.399 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 0057
2025-01-21 11:55:43.748 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_i

In [5]:
print(f"Ignore: {stock_id_ignored}")
print(f"Available: {stock_id_available}")

Ignore: ['0054', '0058', '0059', '0060', '00625K', '00643K', '00649', '00658L', '00659R', '00667', '00672L', '00677U', '00691R', '00698L', '00699R', '00704L', '00705R', '00716R', '00729R', '00732', '00742', '00743', '00766L', '00767', '00774B', '00774C', '00776', '0080', '0081', '008201', '00866', '00906', '00938', '00943', '00944', '00945B', '00946', '00947', '00949', '00951', '00952', '00953B', '00954', '00956', '00960', '00961', '00963', '00964', '00965']
Available: ['0050', '0051', '0052', '0053', '0055', '0056', '0057', '0061', '006203', '006204', '006205', '006206', '006207', '006208', '00631L', '00632R', '00633L', '00634R', '00635U', '00636', '00636K', '00637L', '00638R', '00639', '00640L', '00641R', '00642U', '00643', '00645', '00646', '00647L', '00648R', '00650L', '00651R', '00652', '00653L', '00654R', '00655L', '00656R', '00657', '00657K', '00660', '00661', '00662', '00663L', '00664R', '00665L', '00666R', '00668', '00668K', '00669R', '00670L', '00671R', '00673R', '00674R', '0

In [6]:
start_date="2020-01-01"
end_date="2025-01-01"
trader_fund=1_000_000.0
fee=0.001425
objs = []
candidates = []

for index, row in stock_list_df.iterrows():
    stock_id = row["stock_id"]
    stock_name = row["stock_name"]
    
    if stock_id in stock_id_ignored:
        continue
    else:
        print(stock_id, stock_name)
        
    obj = strategies.BackTest(
         stock_id=stock_id,
         start_date=start_date,
         end_date=end_date,
         trader_fund=trader_fund,
         fee=fee,
         data_loader=api,
    )
    obj.add_strategy(Kd2)
    obj.simulate()
    objs.append(obj)
    
    if index>30: break
    
    time.sleep(1)


2025-01-21 17:10:26.510 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockInfo, data_id: 


0050 元大台灣50


2025-01-21 17:10:35.412 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 0050
2025-01-21 17:10:36.180 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockDividend, data_id: 0050
/home/raul/.config/jupyterlab-desktop/jlab_server/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
2025-01-21 17:10:36.733 | INFO     | FinMind.data.finmind_api:get_data:148 - download TaiwanStockPrice, data_id: TAIEX
2025-01-21 17:10:38.416 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockInfo, data_id: 


0051 元大中型100


2025-01-21 17:10:42.020 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 0051
2025-01-21 17:10:42.538 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockDividend, data_id: 0051
2025-01-21 17:10:42.899 | INFO     | FinMind.data.finmind_api:get_data:148 - download TaiwanStockPrice, data_id: TAIEX
2025-01-21 17:10:44.477 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockInfo, data_id: 


...

2025-01-21 17:12:00.763 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 00638R
2025-01-21 17:12:01.392 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockDividend, data_id: 00638R
/home/raul/.config/jupyterlab-desktop/jlab_server/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
2025-01-21 17:12:01.666 | INFO     | FinMind.data.finmind_api:get_data:148 - download TaiwanStockPrice, data_id: TAIEX
2025-01-21 17:12:03.340 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockInfo, data_id: 


00639 富邦深100


2025-01-21 17:12:03.985 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 00639
2025-01-21 17:12:04.626 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockDividend, data_id: 00639
/home/raul/.config/jupyterlab-desktop/jlab_server/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
2025-01-21 17:12:05.113 | INFO     | FinMind.data.finmind_api:get_data:148 - download TaiwanStockPrice, data_id: TAIEX
2025-01-21 17:12:06.615 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockInfo, data_id: 


00640L 富邦日本正2


2025-01-21 17:12:07.279 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockPrice, data_id: 00640L
2025-01-21 17:12:07.820 | INFO     | FinMind.data.finmind_api:get_data:148 - download Dataset.TaiwanStockDividend, data_id: 00640L
2025-01-21 17:12:08.099 | INFO     | FinMind.data.finmind_api:get_data:148 - download TaiwanStockPrice, data_id: TAIEX


In [31]:
import requests

url = "https://api.web.finmindtrade.com/v2/user_info"
payload = {
    "token": token,
}
resp = requests.get(url, params=payload)
print("使用次數:", resp.json()["user_count"])  # 使用次數
print("api 使用上限", resp.json()["api_request_limit"])  # api 使用上限
print(resp.status_code)
print(resp.json())


使用次數: 0
api 使用上限 600
200
{'msg': 'success', 'status': 200, 'email_verify': True, 'user_id': 'raulhsiao', 'email': 'raulhsiao@gmail.com', 'level': 1, 'end_date': '', 'user_count': 0, 'api_request_limit': 600, 'api_request_limit_hour': 600, 'api_request_limit_day': 600, 'level_title': 'Free', 'is_old_user': 0, 'BackerInfo': {'subscription_begin_date': '', 'subscription_expired_date': '', 'api_request_limit': 0, 'msg': '', 'status_code': 200}, 'SponsorInfo': {'subscription_begin_date': '', 'subscription_expired_date': '', 'api_request_limit': 0, 'msg': '', 'status_code': 200}, 'SponsorProInfo': {'subscription_begin_date': '', 'subscription_expired_date': '', 'api_request_limit': 0, 'msg': '', 'status_code': 200}}


In [1]:
for obj in objs:
    if obj.final_stats.AnnualReturnPer > 5.0:
        print(obj.stock_id, obj.final_stats.AnnualReturnPer)
    elif (obj.final_stats.AnnualReturnPer < 5.0 and obj.final_stats.AnnualReturnPer > 0.0):
        print("\t\t", obj.stock_id, obj.final_stats.AnnualReturnPer)
    else:
        print("\t\t\t\t", obj.stock_id, obj.final_stats.AnnualReturnPer)
obj.s

NameError: name 'objs' is not defined

In [117]:
from bs4 import BeautifulSoup

# 上市清單
# url = "https://www.twse.com.tw/zh/products/securities/etf/products/list.html"
# 國內成分股ETF
url = "https://www.twse.com.tw/zh/products/securities/etf/products/domestic.html"
# 國外成分股ETF(含連結式ETF)
# url = "https://www.twse.com.tw/zh/products/securities/etf/products/foreign.html"
payload = {
}
resp = requests.get(url, params=payload)
bs = BeautifulSoup(resp.content)
print(bs.prettify())

<!DOCTYPE html>
<html lang="zh-Hant-tw">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <title>
   國內成分股ETF - TWSE 臺灣證券交易所
  </title>
  <link href="/favicon.ico" rel="shortcut icon"/>
  <meta content="臺灣證券交易所全球資訊網介紹公司組織、沿革外，並分為「交易資訊」、「上市公司」、 「產品與服務」、「結算服務」、「市場公告」、「法令規章」、「投資人教育」、「統計報表」等，期能提供投資大眾即時完整之訊息，使投資大眾對本公司業務及市場動態有更深一層的瞭解。" name="description"/>
  <meta content="證券,上市,大盤,國際股市指數,匯率,個股行情等,年報,投資,期貨,台灣50,ETF" name="keywords"/>
  <meta content="TWSE" name="author"/>
  <link href="/res/css/web.css" rel="stylesheet"/>
  <meta content="web" name="layout"/>
  <meta content="etf" name="sidebar"/>
  <link href="/res/css/web-report.css" rel="stylesheet"/>
  <style>
   .rwd-table th{min-width:auto;max-width:none}.rwd-table th:nth-child(1){width:8em}.rwd-table small:not(:empty){display:inline-block;font-size:.8em;border-rad

In [74]:
import sys, os

path = "./token"
with open(path, "r") as fp:
    contain = fp.read()
    print(len(contain))

200


In [58]:
help(os.open)

Help on built-in function open in module posix:

open(path, flags, mode=511, *, dir_fd=None)
    Open a file for low level IO.  Returns a file descriptor (integer).

    If dir_fd is not None, it should be a file descriptor open to a directory,
      and path should be relative; path will then be relative to that directory.
    dir_fd may not be implemented on your platform.
      If it is unavailable, using it will raise a NotImplementedError.

